* LLM used: Meta Llama3 8b Instruct LLM model
* Embedding used: HuggingFace BGE Large EN Embedding model
* Dataset used: TheProcessorFood (HuggingFace)

In [1]:
%%writefile requirements.txt
langchain==0.1.14
pypdf==4.1.0
faiss-cpu==1.8.0
boto3==1.34.58
sqlalchemy==2.0.29

Overwriting requirements.txt


In [2]:
import sqlalchemy
print(sqlalchemy.__version__)

2.0.29


In [3]:
pip install nvidia-ml-py3==7.352.0

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install sqlparse==0.5.0

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install scikit-learn==1.3.0

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install omegaconf==2.2.3

Note: you may need to restart the kernel to use updated packages.


In [7]:
pip install gluonts==0.15.1

Note: you may need to restart the kernel to use updated packages.


In [8]:
pip install langchain==0.1.14

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install --upgrade langchain langchain-core

  Using cached langchain-0.3.7-py3-none-any.whl.metadata (7.1 kB)
  Using cached langchain_core-0.3.15-py3-none-any.whl.metadata (6.3 kB)
  Using cached langchain_text_splitters-0.3.2-py3-none-any.whl.metadata (2.3 kB)
  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
  Using cached pydantic_core-2.23.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.6 kB)
Using cached langchain-0.3.7-py3-none-any.whl (1.0 MB)
Using cached langchain_core-0.3.15-py3-none-any.whl (408 kB)
Using cached langchain_text_splitters-0.3.2-py3-none-any.whl (25 kB)
Using cached pydantic-2.9.2-py3-none-any.whl (434 kB)
Using cached pydantic_core-2.23.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (2.1 MB)
  Attempting uninstall: pydantic-core
    Found existing installation: pydantic_core 2.18.4
    Uninstalling pydantic_core-2.18.4:
      Successfully uninstalled pydantic_core-2.18.4
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.7.

In [10]:
pip install boto3==1.34.58

Note: you may need to restart the kernel to use updated packages.


In [11]:
!pip install -U -r requirements.txt

  Using cached langchain-0.1.14-py3-none-any.whl.metadata (13 kB)
  Using cached SQLAlchemy-2.0.29-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (9.6 kB)
  Using cached langchain_core-0.1.53-py3-none-any.whl.metadata (5.9 kB)
  Using cached langchain_text_splitters-0.0.2-py3-none-any.whl.metadata (2.2 kB)
Using cached langchain-0.1.14-py3-none-any.whl (812 kB)
Using cached SQLAlchemy-2.0.29-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.2 MB)
Using cached langchain_core-0.1.53-py3-none-any.whl (303 kB)
Using cached langchain_text_splitters-0.0.2-py3-none-any.whl (23 kB)
  Attempting uninstall: sqlalchemy
    Found existing installation: SQLAlchemy 2.0.32
    Uninstalling SQLAlchemy-2.0.32:
      Successfully uninstalled SQLAlchemy-2.0.32
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.15
    Uninstalling langchain-core-0.3.15:
      Successfully uninstalled langchain-core-0.3.15
  Attempting uninstall: lang

In [12]:
import langchain
print(langchain.__version__)

0.1.14


In [13]:
try:
    import sagemaker
except ImportError:
    !pip install sagemaker

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml


In [14]:
# Import the JumpStartModel class from the SageMaker JumpStart library
from sagemaker.jumpstart.model import JumpStartModel

In [15]:
# Specify the model ID for the HuggingFace Llama 3 8b Instruct LLM model
model_id = "meta-textgeneration-llama-3-8b-instruct"
accept_eula = True
model = JumpStartModel(model_id=model_id, model_version="2.7.0", instance_type= "ml.g5.2xlarge")

In [16]:
predictor = model.deploy(accept_eula=accept_eula, instance_type="ml.g5.2xlarge")

-----------------!

In [17]:
# Specify the model ID for the HuggingFace BGE Large EN Embedding model
model_id = "huggingface-sentencesimilarity-bge-large-en-v1-5"
text_embedding_model = JumpStartModel(model_id=model_id,model_version="1.1.1")

In [18]:
embedding_predictor = text_embedding_model.deploy(instance_type="ml.g5.2xlarge")

---------!

In [19]:
import json
import sagemaker

from langchain_core.prompts import PromptTemplate
from langchain_community.llms import SagemakerEndpoint
from langchain_community.embeddings import SagemakerEndpointEmbeddings
from langchain_community.llms.sagemaker_endpoint import LLMContentHandler
from langchain_community.embeddings.sagemaker_endpoint import EmbeddingsContentHandler

In [20]:
sess = sagemaker.session.Session()
region = sess._region_name

In [21]:
llm_endpoint_name = "meta-textgeneration-llama-3-8b-instruct-2024-11-06-05-03-48-896"
embedding_endpoint_name = "hf-sentencesimilarity-bge-large-en-v1-5-2024-11-06-05-13-34-625"


In [22]:
# testing out my endpoint
import boto3
runtime_client = boto3.client('sagemaker-runtime', region_name=region)

input_prompt = {
    "inputs": "Where is the capital of China?"
}

response = runtime_client.invoke_endpoint(
    EndpointName=llm_endpoint_name,
    ContentType='application/json',
    Body=json.dumps(input_prompt)
)

In [24]:
# read and print the output
output = json.loads(response['Body'].read().decode())
print("LLM Response:", output)

LLM Response: {'generated_text': ' The capital of China is Beijing. Beijing is located in the northern part of the country and is the second most populous city in China, with a population'}


In [23]:
from typing import Dict

class Llama38BContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs: dict) -> bytes:
        payload = {
            "inputs": prompt,
            # default parameters
            "parameters": {
                # Controls the maximum number of tokens the model can generate
                "max_new_tokens": 2000,
                # lower value makes the model more deterministic / higher value allows for more diverse responses
                "top_p": 0.95,
                # randomness
                "temperature": 0.8,
                "stop": ["<|eot_id|>"],
            },
        }
        input_str = json.dumps(
            payload,
        )
        print(input_str)
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        print(response_json)
        content = response_json["generated_text"].strip()
        return content

In [24]:
# Instantiate the content handler for Llama3-8B
llama_content_handler = Llama38BContentHandler()

# Setup for using the Llama3-8B model with SageMaker Endpoint
llm = SagemakerEndpoint(
     endpoint_name=llm_endpoint_name,
     region_name=region,
     model_kwargs={"max_new_tokens": 2000, "top_p": 0.9, "temperature": 0.8},
     content_handler=llama_content_handler
 )

In [25]:
from typing import List

class BGEContentHandlerV15(EmbeddingsContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, text_inputs: List[str], model_kwargs: dict) -> bytes:
        """
        Transforms the input into bytes that can be consumed by SageMaker endpoint.
        Args:
            text_inputs (list[str]): A list of input text strings to be processed.
            model_kwargs (Dict): Additional keyword arguments to be passed to the endpoint.
               Possible keys and their descriptions:
               - mode (str): Inference method. Valid modes are 'embedding', 'nn_corpus', and 'nn_train_data'.
               - corpus (str): Corpus for Nearest Neighbor. Required when mode is 'nn_corpus'.
               - top_k (int): Top K for Nearest Neighbor. Required when mode is 'nn_corpus'.
               - queries (list[str]): Queries for Nearest Neighbor. Required when mode is 'nn_corpus' or 'nn_train_data'.
        Returns:
            The transformed bytes input.
        """
        input_str = json.dumps(
            {
                "text_inputs": text_inputs,
                **model_kwargs
            }
        )
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> List[List[float]]:
        """
        Transforms the bytes output from the endpoint into a list of embeddings.
        Args:
            output: The bytes output from SageMaker endpoint.
        Returns:
            The transformed output - list of embeddings
        Note:
            The length of the outer list is the number of input strings.
            The length of the inner lists is the embedding dimension.
        """
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json["embedding"]

In [26]:
bge_content_handler = BGEContentHandlerV15()
sagemaker_embeddings = SagemakerEndpointEmbeddings(
    endpoint_name=embedding_endpoint_name,
    region_name=region,
    model_kwargs={"mode": "embedding"},
    content_handler=bge_content_handler,
)

In [27]:
import pandas as pd
# Load data (part of HuggingFace TheStoreProcessor dataset)
df = pd.read_csv("new_recipe_part1.csv")
print("CSV columns:", df.columns.tolist())
print("\nFirst row of data:")
print(df.iloc[0])

CSV columns: ['title_cleaned', 'recipe_new', 'allergy_type', 'diet_type', 'holiday', 'cuisine_type', 'meal_type', 'ingredients_alternatives']

First row of data:
title_cleaned                             Roasted Vegetables with Pecan Pesto
recipe_new                  Roasted Vegetables with Pecan Pesto\n1 1/2 pou...
allergy_type                                       [Nut, Beet, Fruit, Cheese]
diet_type                   [(Vegetarian,80),(Gluten-Free,70),(Dairy-Free,...
holiday                                                          Thanksgiving
cuisine_type                                                American, Italian
meal_type                                                   Dinner, Side dish
ingredients_alternatives    Carrots: sweet potatoes, turnips, rutabaga, ra...
Name: 0, dtype: object


In [28]:
import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

# Initialize an empty list to hold the documents
documents = []

# Convert DataFrame rows to documents
for idx, row in df.iterrows():
    # Combine relevant information from your CSV into a text string
    # Adjust this based on what content you want to process
    text_content = f"""
    Title: {row['title_cleaned']}
    Recipe: {row['recipe_new']}
    Allergy Type: {row['allergy_type']}
    Diet Type: {row['diet_type']}
    Holiday: {row['holiday']}
    Cuisine Type: {row['cuisine_type']}
    Meal Type: {row['meal_type']}
    Ingredients Alternatives: {row['ingredients_alternatives']}
    """
    
    # Create a Document object with the text and metadata
    doc = Document(
        page_content=text_content,
        metadata={"source": f"recipe_{idx}", "title": row['title_cleaned']}
    )
    documents.append(doc)

# Set a chunk size for splitting documents
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=2048,
    chunk_overlap=102,
)

# Split the documents
docs = text_splitter.split_documents(documents)

# Print a sample document to verify
if len(docs) > 50:
    print(docs[50])
else:
    print(f"Total documents created: {len(docs)}")
    print("Sample first document:")
    print(docs[0])

page_content="Instructions:\n1. In a saucepan, heat oil, onions, and peppers over medium heat. Simmer for 7 minutes.\n2. In a large bowl, combine egg yolks, lime juice, mustard, hot pepper sauce, fish sauce, and white pepper.\n3. Slowly pour the yolk mixture into the oil while whisking constantly.\n4. Return the bowl to the heat for 30 seconds and whisk constantly to ensure the sauce is fully incorporated.\n5. Set the sauce down over a bowl of ice and whisk to cool quickly. Refrigerate until well chilled.\n6. Pick over crabmeat to remove any shell pieces.\n7. Add herbs and gently toss without breaking up the crab.\n8. Add two or three heaping tablespoons of the chilled sauce to the crab and gently combine.\n9. Add enough breadcrumbs to bind the crab mixture.\n10. Check the mixture for moisture and add more sauce or breadcrumbs as needed.\n11. Spread Japanese breadcrumbs on a work surface and place a ring mold on top.\n12. Pat the crab into the mold and top with more Japanese breadcrumb

In [29]:
avg_doc_length = lambda documents: sum([len(doc.page_content) for doc in documents])//len(documents)

print(f'Average length among {len(documents)} documents loaded is {avg_doc_length(documents)} characters.')
print(f'After the split we have {len(docs)} documents as opposed to the original {len(documents)}.')
print(f'Average length among {len(docs)} documents (after split) is {avg_doc_length(docs)} characters.')

Average length among 7446 documents loaded is 1954 characters.
After the split we have 11419 documents as opposed to the original 7446.
Average length among 11419 documents (after split) is 1273 characters.


In [32]:
sample_embedding = np.array(sagemaker_embeddings.embed_query(docs[0].page_content))
print("Sample embedding of a document chunk: ", sample_embedding)
print("Size of the embedding: ", sample_embedding.shape)

Sample embedding of a document chunk:  [ 0.01922879 -0.01651302  0.00607892 ... -0.04749012  0.03406158
  0.00760758]
Size of the embedding:  (1024,)


In [33]:
# using FAISS for building a vector store
from langchain_community.vectorstores import FAISS
from langchain.indexes.vectorstore import VectorStoreIndexWrapper

vectorstore_faiss = FAISS.from_documents(
    docs,
    sagemaker_embeddings,
)
wrapper_store_faiss = VectorStoreIndexWrapper(vectorstore=vectorstore_faiss)

In [34]:
prompt_template = """You are a professional chef and certified nutritionist specializing in food allergies and dietary restrictions. Generate {num_recipes} COMPLETELY DIFFERENT recipes based on the user's request.

SAFETY & REQUIREMENTS CHECK:
1. Allergens & Diet:
   - List all identified allergens and sensitivities
   - Religious/cultural dietary restrictions
   - Additional dietary preferences (e.g., low-sodium)
   - Cross-contamination concerns

2. Recipe Differentiation:
   Each recipe must have unique:
   - Cooking method (e.g., stir-fry, baked, grilled)
   - Main protein/ingredient source
   - Flavor profile and texture
   - Base seasonings and sauces

FOR EACH RECIPE PROVIDE:

1. Overview:
   - Title (with cultural origin if applicable)
   - Difficulty level (Easy/Medium/Hard)
   - Time (Prep/Cook/Total)
   - Servings
   - Comprehensive allergen and dietary compliance statement

2. Ingredients:
   - Full ingredient list with quantities
   - Clear allergen markings
   - 2-3 substitution options for main ingredients
   - Cross-contamination prevention notes

3. Instructions:
   - Clear, numbered steps
   - Critical technique notes
   - Safety measures for allergen handling
   - Common mistakes to avoid

4. Storage & Serving:
   - Plating suggestions
   - Storage guidelines
   - Safe reheating instructions

SAFETY NOTICE:
- Confirm allergen compliance
- Note cross-contamination risks
- Recommend healthcare provider consultation for severe allergies

{query}
"""

PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["query", "num_recipes"]
)

In [35]:
# Function to get multiple recipes
def get_multiple_recipes(llm, query, num_recipes):
    recipes = []
    try:
        for i in range(num_recipes):
            modified_query = f"""Give me recipe #{i+1} for: {query} 
            Important: This recipe must be COMPLETELY DIFFERENT in style, preparation method, and ingredients from any previous recipes."""
            
            response = llm.invoke(
                PROMPT.format(
                    query=modified_query,
                    num_recipes="1"
                )
            )
            
            formatted_response = f"\n=== Recipe {i+1} ===\n{response}"
            recipes.append(formatted_response)
        
        return "\n\n".join(recipes)
    except Exception as e:
        print(f"Detailed error: {str(e)}")
        return None

# Allergy = Nut & Treenut / cousine = Thai
try:
    query = "I am allergic to nuts. Can you give me a Thai egg noodle recipe free of nuts?"
    multi_recipes = get_multiple_recipes(llm, query, 3)
    print(multi_recipes)
except Exception as e:
    print(f"Error: {e}")

{"inputs": "You are a professional chef and certified nutritionist specializing in food allergies and dietary restrictions. Generate 1 COMPLETELY DIFFERENT recipes based on the user's request.\n\nSAFETY & REQUIREMENTS CHECK:\n1. Allergens & Diet:\n   - List all identified allergens and sensitivities\n   - Religious/cultural dietary restrictions\n   - Additional dietary preferences (e.g., low-sodium)\n   - Cross-contamination concerns\n\n2. Recipe Differentiation:\n   Each recipe must have unique:\n   - Cooking method (e.g., stir-fry, baked, grilled)\n   - Main protein/ingredient source\n   - Flavor profile and texture\n   - Base seasonings and sauces\n\nFOR EACH RECIPE PROVIDE:\n\n1. Overview:\n   - Title (with cultural origin if applicable)\n   - Difficulty level (Easy/Medium/Hard)\n   - Time (Prep/Cook/Total)\n   - Servings\n   - Comprehensive allergen and dietary compliance statement\n\n2. Ingredients:\n   - Full ingredient list with quantities\n   - Clear allergen markings\n   - 2-

In [41]:
# Function to get multiple recipes
def get_multiple_recipes(llm, query, num_recipes):
    recipes = []
    try:
        for i in range(num_recipes):
            modified_query = f"""Give me recipe #{i+1} for: {query} 
            Important: This recipe must be COMPLETELY DIFFERENT in style, preparation method, and ingredients from any previous recipes."""
            
            response = llm.invoke(
                PROMPT.format(
                    query=modified_query,
                    num_recipes="1"
                )
            )
            
            formatted_response = f"\n=== Recipe {i+1} ===\n{response}"
            recipes.append(formatted_response)
        
        return "\n\n".join(recipes)
    except Exception as e:
        print(f"Detailed error: {str(e)}")
        return None

# Allergy = Gluten (Wheat) / cousine = Italian
try:
    query = "I have celiac disease. Please suggest an Italian polenta-based recipe that's naturally gluten-free."
    multi_recipes = get_multiple_recipes(llm, query, 3)
    print(multi_recipes)
except Exception as e:
    print(f"Error: {e}")

{"inputs": "You are a professional chef and certified nutritionist specializing in food allergies and dietary restrictions. Generate 1 COMPLETELY DIFFERENT recipes based on the user's request.\n\nSAFETY & REQUIREMENTS CHECK:\n1. Allergens & Diet:\n   - List all identified allergens and sensitivities\n   - Religious/cultural dietary restrictions\n   - Additional dietary preferences (e.g., low-sodium)\n   - Cross-contamination concerns\n\n2. Recipe Differentiation:\n   Each recipe must have unique:\n   - Cooking method (e.g., stir-fry, baked, grilled)\n   - Main protein/ingredient source\n   - Flavor profile and texture\n   - Base seasonings and sauces\n\nFOR EACH RECIPE PROVIDE:\n\n1. Overview:\n   - Title (with cultural origin if applicable)\n   - Difficulty level (Easy/Medium/Hard)\n   - Time (Prep/Cook/Total)\n   - Servings\n   - Comprehensive allergen and dietary compliance statement\n\n2. Ingredients:\n   - Full ingredient list with quantities\n   - Clear allergen markings\n   - 2-

In [42]:
# Function to get multiple recipes
def get_multiple_recipes(llm, query, num_recipes):
    recipes = []
    try:
        for i in range(num_recipes):
            modified_query = f"""Give me recipe #{i+1} for: {query} 
            Important: This recipe must be COMPLETELY DIFFERENT in style, preparation method, and ingredients from any previous recipes."""
            
            response = llm.invoke(
                PROMPT.format(
                    query=modified_query,
                    num_recipes="1"
                )
            )
            
            formatted_response = f"\n=== Recipe {i+1} ===\n{response}"
            recipes.append(formatted_response)
        
        return "\n\n".join(recipes)
    except Exception as e:
        print(f"Detailed error: {str(e)}")
        return 

# Allergy = Milk & Egg / cousine = Dessert
try:
    query = "Looking for an ice cream alternative - I'm allergic to milk and eggs. Can you suggest a frozen dessert recipe?"
    multi_recipes = get_multiple_recipes(llm, query, 3)
    print(multi_recipes)
except Exception as e:
    print(f"Error: {e}")

{"inputs": "You are a professional chef and certified nutritionist specializing in food allergies and dietary restrictions. Generate 1 COMPLETELY DIFFERENT recipes based on the user's request.\n\nSAFETY & REQUIREMENTS CHECK:\n1. Allergens & Diet:\n   - List all identified allergens and sensitivities\n   - Religious/cultural dietary restrictions\n   - Additional dietary preferences (e.g., low-sodium)\n   - Cross-contamination concerns\n\n2. Recipe Differentiation:\n   Each recipe must have unique:\n   - Cooking method (e.g., stir-fry, baked, grilled)\n   - Main protein/ingredient source\n   - Flavor profile and texture\n   - Base seasonings and sauces\n\nFOR EACH RECIPE PROVIDE:\n\n1. Overview:\n   - Title (with cultural origin if applicable)\n   - Difficulty level (Easy/Medium/Hard)\n   - Time (Prep/Cook/Total)\n   - Servings\n   - Comprehensive allergen and dietary compliance statement\n\n2. Ingredients:\n   - Full ingredient list with quantities\n   - Clear allergen markings\n   - 2-

In [43]:
# Function to get multiple recipes
def get_multiple_recipes(llm, query, num_recipes):
    recipes = []
    try:
        for i in range(num_recipes):
            modified_query = f"""Give me recipe #{i+1} for: {query} 
            Important: This recipe must be COMPLETELY DIFFERENT in style, preparation method, and ingredients from any previous recipes."""
            
            response = llm.invoke(
                PROMPT.format(
                    query=modified_query,
                    num_recipes="1"
                )
            )
            
            formatted_response = f"\n=== Recipe {i+1} ===\n{response}"
            recipes.append(formatted_response)
        
        return "\n\n".join(recipes)
    except Exception as e:
        print(f"Detailed error: {str(e)}")
        return None

# Allergy = Soy / cousine = Vegan
try:
    query = "I am allergic to soy and eat vegan. Can you share a recipe for a protein-rich main dish using legumes instead of soy products?"
    multi_recipes = get_multiple_recipes(llm, query, 3)
    print(multi_recipes)
except Exception as e:
    print(f"Error: {e}")

{"inputs": "You are a professional chef and certified nutritionist specializing in food allergies and dietary restrictions. Generate 1 COMPLETELY DIFFERENT recipes based on the user's request.\n\nSAFETY & REQUIREMENTS CHECK:\n1. Allergens & Diet:\n   - List all identified allergens and sensitivities\n   - Religious/cultural dietary restrictions\n   - Additional dietary preferences (e.g., low-sodium)\n   - Cross-contamination concerns\n\n2. Recipe Differentiation:\n   Each recipe must have unique:\n   - Cooking method (e.g., stir-fry, baked, grilled)\n   - Main protein/ingredient source\n   - Flavor profile and texture\n   - Base seasonings and sauces\n\nFOR EACH RECIPE PROVIDE:\n\n1. Overview:\n   - Title (with cultural origin if applicable)\n   - Difficulty level (Easy/Medium/Hard)\n   - Time (Prep/Cook/Total)\n   - Servings\n   - Comprehensive allergen and dietary compliance statement\n\n2. Ingredients:\n   - Full ingredient list with quantities\n   - Clear allergen markings\n   - 2-

In [44]:
# Function to get multiple recipes
def get_multiple_recipes(llm, query, num_recipes):
    recipes = []
    try:
        for i in range(num_recipes):
            modified_query = f"""Give me recipe #{i+1} for: {query} 
            Important: This recipe must be COMPLETELY DIFFERENT in style, preparation method, and ingredients from any previous recipes."""
            
            response = llm.invoke(
                PROMPT.format(
                    query=modified_query,
                    num_recipes="1"
                )
            )
            
            formatted_response = f"\n=== Recipe {i+1} ===\n{response}"
            recipes.append(formatted_response)
        
        return "\n\n".join(recipes)
    except Exception as e:
        print(f"Detailed error: {str(e)}")
        return 

# Allergy = Fish & Crustacean / cousine = Japanese
try:
    query = "I have seafood allergies (fish and shellfish). Can you provide a recipe for vegetable tempura with a dipping sauce that doesn't contain dashi or fish sauce?"
    multi_recipes = get_multiple_recipes(llm, query, 3)
    print(multi_recipes)
except Exception as e:
    print(f"Error: {e}")

{"inputs": "You are a professional chef and certified nutritionist specializing in food allergies and dietary restrictions. Generate 1 COMPLETELY DIFFERENT recipes based on the user's request.\n\nSAFETY & REQUIREMENTS CHECK:\n1. Allergens & Diet:\n   - List all identified allergens and sensitivities\n   - Religious/cultural dietary restrictions\n   - Additional dietary preferences (e.g., low-sodium)\n   - Cross-contamination concerns\n\n2. Recipe Differentiation:\n   Each recipe must have unique:\n   - Cooking method (e.g., stir-fry, baked, grilled)\n   - Main protein/ingredient source\n   - Flavor profile and texture\n   - Base seasonings and sauces\n\nFOR EACH RECIPE PROVIDE:\n\n1. Overview:\n   - Title (with cultural origin if applicable)\n   - Difficulty level (Easy/Medium/Hard)\n   - Time (Prep/Cook/Total)\n   - Servings\n   - Comprehensive allergen and dietary compliance statement\n\n2. Ingredients:\n   - Full ingredient list with quantities\n   - Clear allergen markings\n   - 2-

In [45]:
import pandas as pd

def generate_recipes():
    generated_recipes = []
    recipes_dict_list = []  # For CSV structure
    
    # List of scenarios with their queries
    scenarios = [
        "I am allergic to nuts and tree nuts. Can you give me a Thai curry recipe free of nuts?",
        "I am allergic to wheat (gluten). Can you give me an Italian chicken recipe that's gluten-free?",
        "I am allergic to milk and eggs. Can you give me a dessert recipe free from these allergens?",
        "I am allergic to soy. Can you give me a vegan taco recipe without soy?",
        "I am allergic to fish and shellfish. Can you give me a Japanese dumpling recipe free from seafood?"
    ]
    
    for scenario_idx, query in enumerate(scenarios, 1):
        # Generate 3 recipes for each scenario
        for recipe_num in range(1, 4):
            try:
                response = llm.invoke(
                    PROMPT.format(
                        query=f"Give me recipe #{recipe_num} for: {query} Important: This recipe must be COMPLETELY DIFFERENT in style, preparation method, and ingredients from any previous recipes.",
                        num_recipes="1"
                    )
                )
                
                # Store complete response for text file
                generated_recipes.append(f"\n=== Scenario {scenario_idx}, Recipe {recipe_num} ===\n")
                generated_recipes.append(f"Query: {query}\n")
                generated_recipes.append(response)
                generated_recipes.append("\n" + "="*50 + "\n")
                
                # Parse for CSV structure
                recipe_dict = {
                    'scenario': f"Scenario {scenario_idx}",
                    'query': query,
                    'recipe_number': f"Recipe {recipe_num}",
                    'full_recipe': response
                }
                recipes_dict_list.append(recipe_dict)
                
            except Exception as e:
                print(f"Error in Scenario {scenario_idx}, Recipe {recipe_num}: {e}")
                continue
    
    # Save as text file
    with open('generated_recipes.txt', 'w') as f:
        f.write('\n'.join(generated_recipes))
    
    # Save as CSV
    #df = pd.DataFrame(recipes_dict_list)
    #df.to_csv('generated_recipes.csv', index=False)
    
    return generated_recipes

# Run generation silently
try:
    _ = generate_recipes()
except Exception as e:
    print(f"Error in recipe generation: {e}")

{"inputs": "You are a professional chef and certified nutritionist specializing in food allergies and dietary restrictions. Generate 1 COMPLETELY DIFFERENT recipes based on the user's request.\n\nSAFETY & REQUIREMENTS CHECK:\n1. Allergens & Diet:\n   - List all identified allergens and sensitivities\n   - Religious/cultural dietary restrictions\n   - Additional dietary preferences (e.g., low-sodium)\n   - Cross-contamination concerns\n\n2. Recipe Differentiation:\n   Each recipe must have unique:\n   - Cooking method (e.g., stir-fry, baked, grilled)\n   - Main protein/ingredient source\n   - Flavor profile and texture\n   - Base seasonings and sauces\n\nFOR EACH RECIPE PROVIDE:\n\n1. Overview:\n   - Title (with cultural origin if applicable)\n   - Difficulty level (Easy/Medium/Hard)\n   - Time (Prep/Cook/Total)\n   - Servings\n   - Comprehensive allergen and dietary compliance statement\n\n2. Ingredients:\n   - Full ingredient list with quantities\n   - Clear allergen markings\n   - 2-

In [46]:
import json

def generate_recipes():
    recipes_database = {
        'scenarios': []
    }
    
    scenarios = [
        "I am allergic to nuts and tree nuts. Can you give me a Thai curry recipe free of nuts?",
        "I am allergic to wheat (gluten). Can you give me an Italian chicken recipe that's gluten-free?",
        "I am allergic to milk and eggs. Can you give me a dessert recipe free from these allergens?",
        "I am allergic to soy. Can you give me a vegan taco recipe without soy?",
        "I am allergic to fish and shellfish. Can you give me a Japanese dumpling recipe free from seafood?"
    ]
    
    for scenario_idx, query in enumerate(scenarios, 1):
        scenario_data = {
            'scenario_id': scenario_idx,
            'query': query,
            'recipes': []
        }
        
        for recipe_num in range(1, 4):
            try:
                response = llm.invoke(
                    PROMPT.format(
                        query=f"Give me recipe #{recipe_num} for: {query} Important: This recipe must be COMPLETELY DIFFERENT in style, preparation method, and ingredients from any previous recipes.",
                        num_recipes="1"
                    )
                )
                
                # Parse recipe into structured data
                recipe_data = {
                    'recipe_id': recipe_num,
                    'title': '',
                    'allergen_info': '',
                    'cuisine_authenticity': '',
                    'ingredients': '',
                    'instructions': '',
                    'serving_storage': '',
                    'safety_check': ''
                }
                
                # Parse the response into sections
                sections = response.split('**')
                for section in sections:
                    if 'Recipe' in section and 'title' not in section.lower():
                        recipe_data['title'] = section.strip()
                    elif 'Allergen Information' in section:
                        recipe_data['allergen_info'] = section.strip()
                    elif 'Cuisine Authenticity' in section:
                        recipe_data['cuisine_authenticity'] = section.strip()
                    elif 'Ingredients' in section:
                        recipe_data['ingredients'] = section.strip()
                    elif 'Instructions' in section:
                        recipe_data['instructions'] = section.strip()
                    elif 'Serving & Storage' in section:
                        recipe_data['serving_storage'] = section.strip()
                    elif 'Final Safety Check' in section:
                        recipe_data['safety_check'] = section.strip()
                
                scenario_data['recipes'].append(recipe_data)
                
            except Exception as e:
                continue
        
        recipes_database['scenarios'].append(scenario_data)
    
    # Save to JSON file
    with open('recipes_database.json', 'w', encoding='utf-8') as f:
        json.dump(recipes_database, f, indent=2, ensure_ascii=False)
    
    return recipes_database

# Run generation
try:
    recipes_database = generate_recipes()
except Exception as e:
    pass

{"inputs": "You are a professional chef and certified nutritionist specializing in food allergies and dietary restrictions. Generate 1 COMPLETELY DIFFERENT recipes based on the user's request.\n\nSAFETY & REQUIREMENTS CHECK:\n1. Allergens & Diet:\n   - List all identified allergens and sensitivities\n   - Religious/cultural dietary restrictions\n   - Additional dietary preferences (e.g., low-sodium)\n   - Cross-contamination concerns\n\n2. Recipe Differentiation:\n   Each recipe must have unique:\n   - Cooking method (e.g., stir-fry, baked, grilled)\n   - Main protein/ingredient source\n   - Flavor profile and texture\n   - Base seasonings and sauces\n\nFOR EACH RECIPE PROVIDE:\n\n1. Overview:\n   - Title (with cultural origin if applicable)\n   - Difficulty level (Easy/Medium/Hard)\n   - Time (Prep/Cook/Total)\n   - Servings\n   - Comprehensive allergen and dietary compliance statement\n\n2. Ingredients:\n   - Full ingredient list with quantities\n   - Clear allergen markings\n   - 2-